Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

In [1]:
using LinearAlgebra, Statistics

using DataFrames
using BenchmarkTools


In [2]:
using NBInclude
@nbinclude("solve_model.ipynb")

#importou a função solve_tree

In [3]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj, γ
game_parameters = [0.884, 9.84, 0.044, 0.346, 0.2];

In [4]:
# s_W1, j_W1, Pst_array1, Pjt_array1, Ds1, Dj1 = solve_tree(data1, game_parameters);

In [5]:
A1 = solve_tree(data1, game_parameters);

  6.964416 seconds (31.35 M allocations: 707.826 MiB, 3.76% gc time)
  7.036743 seconds (30.02 M allocations: 642.090 MiB, 1.48% gc time)
  8.030065 seconds (30.02 M allocations: 642.090 MiB, 0.88% gc time)
  8.640694 seconds (30.02 M allocations: 642.090 MiB, 1.16% gc time)
  9.230693 seconds (30.02 M allocations: 642.090 MiB, 0.75% gc time)
  9.775619 seconds (30.02 M allocations: 642.090 MiB, 1.01% gc time)
 10.327889 seconds (30.02 M allocations: 642.090 MiB, 0.96% gc time)
 10.230247 seconds (30.02 M allocations: 642.090 MiB, 0.73% gc time)
 10.583783 seconds (30.02 M allocations: 642.090 MiB, 0.91% gc time)
 10.967339 seconds (30.02 M allocations: 642.090 MiB, 0.68% gc time)
 10.850907 seconds (30.02 M allocations: 642.090 MiB, 0.87% gc time)
 10.983497 seconds (30.02 M allocations: 642.090 MiB, 0.63% gc time)
 11.263314 seconds (30.02 M allocations: 642.090 MiB, 0.87% gc time)
 11.163959 seconds (26.96 M allocations: 595.398 MiB, 0.66% gc time)
137.959546 seconds (418.59 M alloc

In [6]:
typeof(A1)

Tuple{Array{Float64,4},Array{Float64,4},Array{Float64,4},Array{Float64,4},Float64,Float64}

In [7]:
# s_W2, j_W2, Pst_array2, Pjt_array2, Ds2, Dj2 = solve_tree(data2, game_parameters);

In [8]:
A2 = solve_tree(data2, game_parameters);

  6.429849 seconds (30.02 M allocations: 642.090 MiB, 1.13% gc time)
  7.669543 seconds (30.02 M allocations: 642.090 MiB, 1.44% gc time)
  8.445905 seconds (30.02 M allocations: 642.090 MiB, 0.87% gc time)
  9.136122 seconds (30.02 M allocations: 642.090 MiB, 1.27% gc time)
  9.339476 seconds (30.02 M allocations: 642.090 MiB, 1.19% gc time)
  9.889460 seconds (30.02 M allocations: 642.090 MiB, 0.73% gc time)
  9.978540 seconds (30.02 M allocations: 642.090 MiB, 1.10% gc time)
 10.169243 seconds (30.02 M allocations: 642.090 MiB, 0.69% gc time)
 10.655863 seconds (30.02 M allocations: 642.090 MiB, 1.06% gc time)
 10.842444 seconds (30.02 M allocations: 642.090 MiB, 1.02% gc time)
 11.077508 seconds (30.02 M allocations: 642.090 MiB, 0.72% gc time)
 11.169468 seconds (26.96 M allocations: 595.398 MiB, 1.02% gc time)
116.185111 seconds (357.18 M allocations: 7.479 GiB, 0.99% gc time)


Simulation parameters

In [9]:
#θs0, θj0, μ
simulation_parameters = [28.0, 36.0, 4.566];

### Criando funções para simular o jogo

In [10]:


# θs0, θj0, μ = simulation_parameters;
# θs0 = Int64(θs0);
# θj0 = Int64(θj0);

# #parameters to some functions that are similar as the ones in solve_model

# c0 = game_parameters[3];
# c1 = Float64(0.0015);

# ρ = game_parameters[1];
# λj = game_parameters[4];


# Vmax1 = data1[1];
# L1 = data1[2];
# D1 = Ds1 + Dj1;


In [11]:
# Vmax2 = data2[1];
# L2 = data2[2];
# D2 = Ds2 + Dj2;

In [12]:
#cost function
function Ct(t, c0=c0, c1=c1)
    
    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function prob_cd(γ, share)

    #probability of a cram down
    return γ * share
end




function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next, pie_share = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2, 3]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
    
    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [13]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == grid)
        
        return grid

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [14]:
function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




# #test
# Hs0 = 30
# Hj0 = 28


# hst = Hs0
# hjt = Hj0

# #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
# lst = hst
# ljt = hjt




# choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [15]:
function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    
    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj
    
    
    c0 = game_parameters[3];
    c1 = Float64(0.0015);

    ρ = game_parameters[1];
    λj = game_parameters[4];
    γ = game_parameters[5];

    
    
#     hst = Hs0
#     hjt = Hj0
    
    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(6);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
        
        #proposal ####
        policy, payoff_prop, payment, lm_next, pie_share = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                if(u < prob_cd(γ, pie_share)) #cram down occurs
                    
                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals
                    result[6] = one(1.0) #dummy equal to one if cram down occurs
                else
                    
                    t+=1

                    hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
                    
                end

            end

        elseif(policy==2.0)
            
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
            
            

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [16]:
#unit testing
simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

6-element Array{Float64,1}:
 0.20521288730015472
 0.05208036862789612
 3.0
 5.0
 5.0
 0.0

In [17]:
#unit testing 
simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])

6-element Array{Float64,1}:
 0.44886016859757355
 0.4196575714761128
 3.0
 6.0
 6.0
 0.0

In [18]:
function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    
    
    
    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster
    
    
    
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 8)

    #últimas entradas de Results são os valores das dívidas
    
    
    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:6] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:6] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end
            
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [19]:
#unit testing
S = 40
N = 66
Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

40×66×8 Array{Float64,3}:
[:, :, 1] =
 0.334391  0.167653  0.202538  0.195678  …  0.508065  0.270641  0.990482
 0.211746  0.259264  0.201947  0.219036     0.395333  0.500587  0.290358
 0.230535  0.247593  0.170074  0.3148       0.538314  0.407365  0.427067
 0.211746  0.211746  0.248959  0.237461     0.514959  0.544685  0.298699
 0.211746  0.176549  0.248898  0.326414     0.477867  0.463134  0.746761
 0.247331  0.18891   0.207485  0.227717  …  0.422092  0.459543  0.428834
 0.161471  0.21315   0.192443  0.211746     0.436532  0.462185  0.525446
 0.195044  0.236745  0.216509  0.20567      0.498675  0.431933  0.633466
 0.247974  0.245565  0.253284  0.197162     0.377391  0.58525   0.358142
 0.22954   0.231914  0.247249  0.168242     0.507828  0.573711  0.339081
 0.271737  0.191578  0.211746  0.275665  …  0.478965  0.501783  0.791219
 0.191112  0.252443  0.195937  0.218003     0.623239  0.26      0.596912
 0.211746  0.277776  0.205007  0.211746     0.533692  0.524564  0.470316
 ⋮           

In [20]:
function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    Moments = zeros(S, 9)
    
    μ = simulation_parameters[3]
    for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.cram_down = Results[s, :, 6];
            DF.Ds = Results[s,:,7];
            DF.Dj = Results[s,:,8];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = size(INCOURT_REORG,1)/ size(INCOURT,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)
        
        
        
            #9. percentage of cases where a cram down occurred inside court
            mm9 = mean(INCOURT.cram_down)
        
            
        
        
            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8, mm9]

            
            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

        end
    return vec(mean(Moments, dims=1))
end



  

simulate_moments (generic function with 1 method)

In [28]:
using Random

# Random.seed!(2)

#test
S = 40
N = 66

simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

9-element Array{Float64,1}:
 1.1630148427036728
 0.9990906084656086
 2.5190612782363075
 0.8147727272727272
 1.1614205347173734
 0.21478075928968252
 0.3965759633734131
 0.5531426813993676
 0.18587773760114998

Calculando a derivada dos momentos em relação aos parâmetros

In [ ]:
#data_moments
data_moments = [0.728, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497];

In [ ]:
function simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(round(θs0, digits=0));
    θj0 = Int64(round(θj0, digits=0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

In [ ]:
# calculando a derivada em relação a ρ

function derivative_moments(S, N, game_parameters, simulation_parameters, simulated_moments, data1, data2, n1)
    
    
    P = size(game_parameters,1)+size(simulation_parameters,1)
    M = size(simulated_moments,1)
    D = zeros(M, P)
    
    estimated_game_parameters = game_parameters
    estimated_simulation_parameters = simulation_parameters
    
    #delta for each parameter
    #approximately 1% or the lowest decimal point
    #[game_parameters, simulation_parameters]
    delta = [0.008, 0.36, 0.02, -0.01, -1.0, 1.0, 0.025]
    
    #PRECISA ALTERAR O DELTA PARA INCLUIR A VARIAÇÃO DE \GAMMA
    
    for i in 1:P
        if(i<=size(game_parameters,1))
            game_parameters = copy(estimated_game_parameters)
            
            game_parameters[i] = game_parameters[i] + delta[i]
            
            
        else
            simulation_parameters = copy(estimated_simulation_parameters)
            simulation_parameters[i-size(game_parameters,1)] = simulation_parameters[i-size(game_parameters,1)] + delta[i]
            
            
        end
        
        
        
        A1 = solve_tree(data1, game_parameters)
        A2 = solve_tree(data2, game_parameters)
        
        new_simulated_moments = simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
        
        D[:, i] = new_simulated_moments - simulated_moments
        
    end
    
    
    
    return D
    
end


In [ ]:
Derivative = derivative_moments(S, N, game_parameters, simulation_parameters, simulated_moments, data1, data2, n1)

In [ ]:
Derivative[2,:]